In [1]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_qdrant import RetrievalMode, QdrantVectorStore
from qdrant_client.models import Filter, FieldCondition, MatchValue

In [2]:
frage = "Wie viele Seiten muss ich in meiner Bachelorarbeit in Wirtschaftsinformatik schreiben?"

In [3]:
encode_kwargs = {'prompt': 'query:'}

In [4]:
embeddings = HuggingFaceEmbeddings(model_name="mixedbread-ai/deepset-mxbai-embed-de-large-v1", encode_kwargs=encode_kwargs)

/Users/bastian/Desktop/BachelorProjektAbgabe/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
mixed_bread_token_collection = "mixedbread-ai_deepset-mxbai-embed-de-large-v1_token_based_chunks"

In [6]:
qdrant_url = "http://localhost:6333"

In [8]:
qdrant_client = QdrantVectorStore.from_existing_collection(
    embedding=embeddings,
    retrieval_mode=RetrievalMode.DENSE,
    prefer_grpc=False,
    collection_name=mixed_bread_token_collection,
    url=qdrant_url,
)

In [7]:
from qdrant_client import QdrantClient, models
client = QdrantClient(url=qdrant_url)
print(client.get_collections())

collections=[CollectionDescription(name='mixedbread-ai_deepset-mxbai-embed-de-large-v1_token_based_chunks'), CollectionDescription(name='intfloat_multilingual-e5-large_token_based_chunks'), CollectionDescription(name='intfloat_multilingual-e5-large_recursive_chunks'), CollectionDescription(name='mixedbread-ai_deepset-mxbai-embed-de-large-v1_recursive_chunks')]


In [8]:
collection_info = client.get_collection(collection_name="mixedbread-ai_deepset-mxbai-embed-de-large-v1_token_based_chunks")
print(collection_info)

status=<CollectionStatus.GREEN: 'green'> optimizer_status=<OptimizersStatusOneOf.OK: 'ok'> vectors_count=None indexed_vectors_count=0 points_count=1763 segments_count=8 config=CollectionConfig(params=CollectionParams(vectors={'': VectorParams(size=1024, distance=<Distance.COSINE: 'Cosine'>, hnsw_config=None, quantization_config=None, on_disk=None, datatype=None, multivector_config=None)}, shard_number=1, sharding_method=None, replication_factor=1, write_consistency_factor=1, read_fan_out_factor=None, on_disk_payload=True, sparse_vectors={}), hnsw_config=HnswConfig(m=16, ef_construct=100, full_scan_threshold=10000, max_indexing_threads=0, on_disk=False, payload_m=None), optimizer_config=OptimizersConfig(deleted_threshold=0.2, vacuum_min_vector_number=1000, default_segment_number=0, max_segment_size=None, memmap_threshold=None, indexing_threshold=20000, flush_interval_sec=5, max_optimization_threads=None), wal_config=WalConfig(wal_capacity_mb=32, wal_segments_ahead=0), quantization_confi

In [9]:
collection_info.config.hnsw_config

HnswConfig(m=16, ef_construct=100, full_scan_threshold=10000, max_indexing_threads=0, on_disk=False, payload_m=None)

In [12]:
client.update_collection(
    collection_name="mixedbread-ai_deepset-mxbai-embed-de-large-v1_token_based_chunks",
    optimizer_config={"indexing_threshold": 20000}
)


True

In [8]:
found_docs = qdrant_client.similarity_search_with_score(
    query=frage,
    k=5
)

In [11]:
found_docs

[(Document(metadata={'id': 1, 'studiengang': 'Wirtschaftsinformatik', 'abschluss': 'Bachelor', 'standort': 'Hagen', 'link': 'https://www.fh-swf.de/de/studienangebot/studiengaenge/wirtschaftsinformatik__b_sc__/wirtschaftsinformatik__b_sc__1.php', 'hash': '52d4e379d2abc0535448fc42b8962da89d850e7865654a3ea8a44648e2549a4a', '_id': '31b36de5-e963-4069-a963-0fee87c5459d', '_collection_name': 'mixedbread-ai_deepset-mxbai-embed-de-large-v1_token_based_chunks'}, page_content='# Wirtschaftsinformatik B.Sc. ## Hagen 1. Studienangebot 2. Studiengänge 3. Hagen 4. Wirtschaftsinformatik (B.Sc.) ## Bachelor-Studium Wirtschaftsinformatik ### Interdisziplinarität! Das ist der große Trumpf des Bachelor-Studiengangs Wirtschaftsinformatik. Er kombiniert Inhalte aus den Bereichen Informatik, Betriebswirtschaftslehre (BWL) und Technik zu einer attraktiven Mischung mit besten beruflichen Perspektiven. Auf dem Lehrplan stehen Fächer wie IT-Sicherheit, Business Intelligence oder Web-Programmierung. Dazu gibt es

In [12]:
aufbau_winfo_frage = "Wie ist der Studiengang Wirtschaftsinformatik aufgebaut?"

In [13]:
found_docs = qdrant_client.similarity_search_with_score(
    query=aufbau_winfo_frage,
    k=5
)

In [14]:
found_docs

[(Document(metadata={'id': 263, 'studiengang': 'Wirtschaftsinformatik', 'abschluss': 'Bachelor', 'standort': 'Meschede', 'link': 'https://www.fh-swf.de/de/studienangebot/studiengaenge/wirtschaftsinformatik_b_sc_/index.php', 'hash': '54a73c342c193961ee06c94b7e5ccc624fc54261ec8ce26e615b4da5f9f59b41', '_id': 'b6a2c3a3-aec2-41ce-936c-98e13af8676f', '_collection_name': 'mixedbread-ai_deepset-mxbai-embed-de-large-v1_token_based_chunks'}, page_content='en sind Wirtschaftsinformatiker*innen bestens ausgestattet für ein breites Spektrum an möglichen beruflichen Entfaltungsmöglichkeiten. Zu den möglichen Berufen zählen: - Für stärker technisch Interessierte: Softwareentwicklung bzw. Anwendung von KI und maschinellem Lernen zur Analyse großer Datenmengen durch Data Science, Entwickler*innen für Softwarearchitektur für betriebliche Anwendung - Berufe an der klassischen Schnittstelle zwischen Wirtschaft und Informatik: Prozessanalysten und IT-Beratung für E-Commerce und anderes E-Business sowie Bus

In [19]:
# Filter nutzen:
metadata_filter = Filter(
    must=[
        FieldCondition(
            key="metadata.standort",
            match=MatchValue(value="Hagen")
        ),
        FieldCondition(
            key="metadata.abschluss",
            match=MatchValue(value="Bachelor")
        ),
        FieldCondition(
            key="metadata.studiengang",
            match=MatchValue(value="Wirtschaftsinformatik")
        )
    ]
)

In [20]:
found_docs = qdrant_client.similarity_search_with_score(
    query=frage,
    k=5,
    filter=metadata_filter
)

In [21]:
found_docs

[(Document(metadata={'id': 21, 'studiengang': 'Wirtschaftsinformatik', 'abschluss': 'Bachelor', 'standort': 'Hagen', 'link': 'https://www.fh-swf.de/media/neu_np/hv_2/prfungsundnderungsordnungen/fbtbw_1/FPO_2018_WiIng_WiInf_IBE_IBI.pdf', 'hash': 'ff861463683837be653a2ba75af2198508491056a31928a7d092453a1ab59a09', '_id': 'a2a4dd4f-eaa3-43ef-8936-aa961484b871', '_collection_name': 'mixedbread-ai_deepset-mxbai-embed-de-large-v1_token_based_chunks'}, page_content='setzung der Gesamtnote in Form eines ECTS gewichteten arithmetischen Mittels der Einzelnoten der durch das entsprechende Learning Agreement vereinbarten Modulprüfungen berücksichtigt. ## Teil 3 ## Das Studium ## § 20 ## Umfang der Bachelorarbeit Der Umfang der Bachelorarbeit gemäß § 28 Absatz 1 RPO beträgt in der Regel 65 Seiten (inklusive Bilder und Tabellen und ohne Verzeichnisse und Anhänge). Die Bearbeitungszeit (Zeitraum von der Ausgabe bis zur Abgabe der Bachelorarbeit) beträgt höchstens zehn Wochen. Auf einen vor Ablauf der 

In [22]:
mixed_bread_recursive_collection = "mixedbread-ai_deepset-mxbai-embed-de-large-v1_recursive_chunks"

In [23]:
qdrant_client = QdrantVectorStore.from_existing_collection(
    embedding=embeddings,
    retrieval_mode=RetrievalMode.DENSE,
    prefer_grpc=False,
    collection_name=mixed_bread_recursive_collection,
    url=qdrant_url,
)

In [26]:
found_docs = qdrant_client.similarity_search_with_score(
    query=frage,
    k=5,
    filter=metadata_filter
)

In [27]:
found_docs

[(Document(metadata={'id': 26, 'studiengang': 'Wirtschaftsinformatik', 'abschluss': 'Bachelor', 'standort': 'Hagen', 'link': 'https://www.fh-swf.de/media/neu_np/hv_2/prfungsundnderungsordnungen/fbtbw_1/FPO_2018_WiIng_WiInf_IBE_IBI.pdf', 'hash': 'ff861463683837be653a2ba75af2198508491056a31928a7d092453a1ab59a09', '_id': '40526838-f368-4298-b977-ac56562ffbaa', '_collection_name': 'mixedbread-ai_deepset-mxbai-embed-de-large-v1_recursive_chunks'}, page_content='## Teil 3\n\n## Das Studium\n\n## § 20\n\n## Umfang der Bachelorarbeit\n\nDer Umfang der Bachelorarbeit gemäß § 28 Absatz 1 RPO beträgt in der Regel 65 Seiten (inklusive Bilder und Tabellen und ohne Verzeichnisse und Anhänge). Die Bearbeitungszeit (Zeitraum  von  der  Ausgabe  bis  zur  Abgabe  der  Bachelorarbeit)  beträgt  höchstens  zehn Wochen. Auf einen vor Ablauf der Frist gestellten begründeten Antrag der Kandidatin oder des Kandidaten kann der Prüfungsausschuss ausnahmsweise eine Nachfrist der Bearbeitungszeit von bis zu drei